In [6]:
import numpy as np 
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt 
from xml.dom import minidom
import csv
import time as tm
import datetime
import tkinter as tk
from tkinter import ttk

root = tk.Tk()
root.title("Group Touch Demo")
root.geometry('990x520')

class TouchPair:
	def __init__(self, orientation, distance, time):
		self.orientation = orientation
		self.distance = distance
		self.time = time

# form a touch pair with 3 calculated features
def pair(touchPoint, lastPoint):
	orientation = round((abs(float(touchPoint[0]) - float(lastPoint[0]))) % 1, 5)
	yDistance = abs(float(touchPoint[1]) - float(lastPoint[1]))
	xDistance = abs(float(touchPoint[2]) - float(lastPoint[2]))
	distance = round(np.sqrt(yDistance*yDistance + xDistance*xDistance) / 1557, 3)
	time = (touchPoint[3] - lastPoint[3])*1000

	paramTouchPair = TouchPair(orientation, distance, time)
	return paramTouchPair

def MLP(paramTouchPair, paramCountPairing):
	classification = predictionResults[paramCountPairing][6]
	# print(paramCountPairing)
	return classification

def demo(event):
	print("demo()")

def drawTouch(paramO, paramY, paramX):
	rLong = 7
	rShort = 5
	paramAnchors = [float(paramX)-rLong, float(paramY)+rShort, float(paramX)+rLong, float(paramY)-rShort]
	return paramAnchors

def GUI():
	frameUserIndicators = tk.Frame(root)
	frameTableTop = tk.Canvas(root, borderwidth=2, relief=tk.GROOVE, width=651, height=419.4)
	frameControlPanel = tk.Frame(root)
	frameUserIndicators.grid(row=0, column=0)
	frameTableTop.grid(row=1, column=0)
	frameControlPanel.grid(row=0, rowspan=2, column=1)

	# user indicators
	indicatorUser1 = tk.Label(frameUserIndicators, text="User 1", anchor=tk.CENTER, 
							padx=10, pady=10, foreground='white', background='green')
	indicatorUser1.grid(row=0, column=0)

	# live demo
	anchors = drawTouch(0,100,100)
	touchTest = frameTableTop.create_oval(anchors[0],anchors[1],anchors[2],anchors[3], fill='green', outline='grey')

	# control panel
	btnStart = tk.Button(frameControlPanel,text="Start", width=10)
	btnStart.grid(row=0, column=0)
	btnStart.bind('<ButtonRelease-1>', demo)

if __name__ == "__main__":
	# read predictions from .csv
	with open("1touchPairsFullInfo.csv", "r") as f:
		reader = csv.reader(f)
		next(reader, None)
		predictionResults = list(reader)
		# prediction = predictionResults[][6]

	# read touch points from .xml
	indexNow = 0
	nameList = ["1 - Brainstorm", 
				"2 - Heuristics", "3 - Heuristics", "4 - Heuristics", "5 - Heuristics", 
				"6 - Map", "7 - Map", "8 - Map", "9 - Map",
				"10 - Resources", "11 - Resources", "12 - Resources", "13 - Resources",
				"14 - Wireframes", "15 - Wireframes", "16 - Wireframes", "17 - Wireframes"]
	xmldoc = minidom.parse(nameList[indexNow]+".xml")
	logs = xmldoc.getElementsByTagName('Point')

	touchPoints = []
	deltaTimeThreshold = 114275 # known from data preprocessing

	countInvalidData = 0
	for i in range(len(logs)):
		if '?' in logs[i].attributes['student'].value and logs[i].attributes['student'].value == "":
			# print("invalid data: missing ground truth")
			countInvalidData += 1
		else:
			if i+countInvalidData == 0:
				tempTimeStr = logs[0].attributes['timestamp'].value
				tempTimeFormat = datetime.datetime.strptime(tempTimeStr, "%Y/%m/%d %H:%M:%S:%f")
				tempTime = tm.mktime(tempTimeFormat.timetuple())+(tempTimeFormat.microsecond/1000000.0)
				thisTime = tempTime
			else:
				thisTimeStr = logs[i].attributes['timestamp'].value
				thisTimeFormat = datetime.datetime.strptime(thisTimeStr, "%Y/%m/%d %H:%M:%S:%f")
				thisTime = tm.mktime(thisTimeFormat.timetuple())+(thisTimeFormat.microsecond/1000000.0)
			deltaTime = (thisTime - tempTime)*1000
			if deltaTime > deltaTimeThreshold:
				# mark this point in grey due to too long time elapsed
				print("invalida data: too long time elapsed")
			else:
				touchPoints.append([logs[i].attributes['o'].value,
									logs[i].attributes['y'].value,
									logs[i].attributes['x'].value,
									thisTime,
									deltaTime])

	# initialize lastPoints with no.1 touchpoint
	# list columns are in order of o, y, x, timestamp, deltaTime, user 
	lastPoints = [[touchPoints[0][0],
					touchPoints[0][1],
					touchPoints[0][2],
					touchPoints[0][3],
					touchPoints[0][4],
					"user1"]]
	countUser = 1
	touchGroup = [[0]] # row: user; column: index of touch point

	countPairing = 0
	for i in range(1, len(touchPoints)):
		# print("---------")
		# print(touchPoints[i])
		flagFirstUser = True
		# flagNewUser = False

		if len(lastPoints) == 1:
			touchPair = pair(touchPoints[i], lastPoints[0])
			classification = MLP(touchPair, countPairing)
			countPairing += 1

			if classification == "same" and flagFirstUser == True:
				touchPoints[i].append("user1") # declare user
				touchGroup[0].append(i)
				lastPoints[0] = touchPoints[i]
				flagFirstUser = False
			elif classification == "different":
				countUser += 1
				touchPoints[i].append("user"+str(countUser))
				touchGroup.append([i])
				lastPoints.append(touchPoints[i])
				# flagNewUser = True

		else:
			countDiff = 0
			for j in range(len(lastPoints)):
				if countPairing <= 12805:
					touchPair = pair(touchPoints[i], lastPoints[j])
					print("countPairing = "+str(countPairing))
					classification = MLP(touchPair, countPairing)
					countPairing += 1

					if classification == "same" and flagFirstUser == True:
						touchPoints[i].append(lastPoints[j][-1]) # declare user
						touchGroup[j].append(i)
						lastPoints[j] = touchPoints[i]
						flagFirstUser = False
						countDiff = 0
						print("same")
					elif classification == "different":
						countDiff += 1
						print("different")
						# print(countDiff)
			if countDiff == len(lastPoints):
				countUser += 1
				touchPoints[i].append("user"+str(countUser))
				touchGroup.append([i])
				lastPoints.append(touchPoints[i])
				# flagNewUser = True
				print("different")

	GUI()
	root.mainloop()

invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
i

invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
i

invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
invalida data: too long time elapsed
i

same
countPairing = 262
different
countPairing = 263
different
countPairing = 264
same
countPairing = 265
different
countPairing = 266
different
countPairing = 267
same
countPairing = 268
different
countPairing = 269
different
countPairing = 270
same
countPairing = 271
different
countPairing = 272
different
countPairing = 273
same
countPairing = 274
different
countPairing = 275
different
countPairing = 276
same
countPairing = 277
different
countPairing = 278
different
countPairing = 279
same
countPairing = 280
different
countPairing = 281
different
countPairing = 282
same
countPairing = 283
different
countPairing = 284
different
countPairing = 285
same
countPairing = 286
different
countPairing = 287
different
countPairing = 288
same
countPairing = 289
different
countPairing = 290
different
countPairing = 291
same
countPairing = 292
different
countPairing = 293
different
countPairing = 294
same
countPairing = 295
different
countPairing = 296
different
countPairing = 297
same
countPairin

different
countPairing = 1252
different
countPairing = 1253
countPairing = 1254
different
countPairing = 1255
different
countPairing = 1256
same
countPairing = 1257
different
countPairing = 1258
different
countPairing = 1259
same
countPairing = 1260
different
countPairing = 1261
different
countPairing = 1262
same
countPairing = 1263
different
countPairing = 1264
different
countPairing = 1265
countPairing = 1266
different
countPairing = 1267
different
countPairing = 1268
same
countPairing = 1269
different
countPairing = 1270
different
countPairing = 1271
same
countPairing = 1272
different
countPairing = 1273
different
countPairing = 1274
same
countPairing = 1275
different
countPairing = 1276
different
countPairing = 1277
countPairing = 1278
different
countPairing = 1279
different
countPairing = 1280
same
countPairing = 1281
different
countPairing = 1282
different
countPairing = 1283
same
countPairing = 1284
different
countPairing = 1285
different
countPairing = 1286
same
countPairing = 

...

countPairing = 3108
different
countPairing = 3109
different
countPairing = 3110
countPairing = 3111
different
countPairing = 3112
different
countPairing = 3113
different
countPairing = 3114
same
countPairing = 3115
different
countPairing = 3116
different
countPairing = 3117
different
countPairing = 3118
same
countPairing = 3119
different
countPairing = 3120
different
countPairing = 3121
different
countPairing = 3122
countPairing = 3123
different
countPairing = 3124
different
countPairing = 3125
different
countPairing = 3126
same
countPairing = 3127
different
countPairing = 3128
different
countPairing = 3129
different
countPairing = 3130
same
countPairing = 3131
different
countPairing = 3132
different
countPairing = 3133
different
countPairing = 3134
countPairing = 3135
different
countPairing = 3136
different
countPairing = 3137
different
countPairing = 3138
same
countPairing = 3139
different
countPairing = 3140
different
countPairing = 3141
different
countPairing = 3142
same
countPairi

countPairing = 4195
different
countPairing = 4196
different
countPairing = 4197
different
countPairing = 4198
different
countPairing = 4199
different
countPairing = 4200
countPairing = 4201
different
countPairing = 4202
different
countPairing = 4203
different
countPairing = 4204
same
countPairing = 4205
different
countPairing = 4206
different
countPairing = 4207
different
countPairing = 4208
countPairing = 4209
different
countPairing = 4210
different
countPairing = 4211
different
countPairing = 4212
same
countPairing = 4213
different
countPairing = 4214
different
countPairing = 4215
different
countPairing = 4216
countPairing = 4217
different
countPairing = 4218
different
countPairing = 4219
different
countPairing = 4220
countPairing = 4221
different
countPairing = 4222
different
countPairing = 4223
different
countPairing = 4224
same
countPairing = 4225
different
countPairing = 4226
different
countPairing = 4227
different
countPairing = 4228
countPairing = 4229
different
countPairing = 

different
countPairing = 5284
same
countPairing = 5285
different
countPairing = 5286
different
countPairing = 5287
different
countPairing = 5288
countPairing = 5289
different
countPairing = 5290
different
countPairing = 5291
different
countPairing = 5292
same
countPairing = 5293
different
countPairing = 5294
different
countPairing = 5295
different
countPairing = 5296
countPairing = 5297
different
countPairing = 5298
different
countPairing = 5299
different
countPairing = 5300
countPairing = 5301
different
countPairing = 5302
different
countPairing = 5303
different
countPairing = 5304
same
countPairing = 5305
different
countPairing = 5306
different
countPairing = 5307
different
countPairing = 5308
countPairing = 5309
different
countPairing = 5310
different
countPairing = 5311
different
countPairing = 5312
same
countPairing = 5313
different
countPairing = 5314
different
countPairing = 5315
different
countPairing = 5316
countPairing = 5317
different
countPairing = 5318
different
countPairi

countPairing = 6364
same
countPairing = 6365
different
countPairing = 6366
different
countPairing = 6367
different
countPairing = 6368
different
countPairing = 6369
different
countPairing = 6370
different
countPairing = 6371
different
countPairing = 6372
same
countPairing = 6373
different
countPairing = 6374
different
countPairing = 6375
different
countPairing = 6376
different
countPairing = 6377
different
countPairing = 6378
different
countPairing = 6379
different
countPairing = 6380
countPairing = 6381
different
countPairing = 6382
different
countPairing = 6383
different
countPairing = 6384
different
countPairing = 6385
different
countPairing = 6386
different
countPairing = 6387
different
countPairing = 6388
same
countPairing = 6389
different
countPairing = 6390
different
countPairing = 6391
different
countPairing = 6392
same
countPairing = 6393
different
countPairing = 6394
different
countPairing = 6395
different
countPairing = 6396
countPairing = 6397
different
countPairing = 6398


different
countPairing = 7533
different
countPairing = 7534
different
countPairing = 7535
different
countPairing = 7536
different
countPairing = 7537
same
countPairing = 7538
different
countPairing = 7539
different
countPairing = 7540
countPairing = 7541
different
countPairing = 7542
different
countPairing = 7543
countPairing = 7544
different
countPairing = 7545
different
countPairing = 7546
different
countPairing = 7547
different
countPairing = 7548
different
countPairing = 7549
same
countPairing = 7550
different
countPairing = 7551
different
countPairing = 7552
different
countPairing = 7553
different
countPairing = 7554
different
countPairing = 7555
countPairing = 7556
different
countPairing = 7557
different
countPairing = 7558
same
countPairing = 7559
different
countPairing = 7560
different
countPairing = 7561
countPairing = 7562
different
countPairing = 7563
different
countPairing = 7564
different
countPairing = 7565
different
countPairing = 7566
different
countPairing = 7567
diffe

In [7]:
countPairing

8109

In [8]:
len(touchGroup)

12

In [9]:
touchPoints[1000]

['0.324203', '205', '770.667', 1404904503.985, 61531.999826431274, 'user4']